# Helper Functions:
- connect_2_api() 
- refresh_token()
- save_to_folder(dataframe, 'specific_path_and_name_string', 'separator')

### V3 Updates / changes

- 1.  Pulling most of the extraneous data out of the Python data request loop (now feeding it the list of SP500)
- 2. Will explore saving company data in dictionary format vs in pandas datframe at first (column / pivoting issue with the financial data)
- 3. Will open and explore my UFL data to align on the basic dataset I will run my first models with

## Login and Refresh
- note: avoid turning connecting to api into a function as it is difficult to change variables needed for any api call

In [1]:
import os, re, sys, json
import requests
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import getpass
from datetime import datetime
import urllib
from urllib.parse import urlencode

In [2]:
# Definining my email, pw, client id and secret here so I can use it here and in API pull cell below  
email = 'xxxxxxxx@xxxxx.com'
password = 'XXXYYY'
client_id = 'XXXXXXXXXXXXXXXXX'
client_secret = 'XXXXXXXXXXXXXXXXX'

    #def connect_2_api():  #Now I can just login in a cell using connect_2_api()
body_auth = {'username' : email,
            'client_id': client_id,
            'client_secret' : client_secret,
            'password' : password,
            'grant_type' : 'password',
            'platform' : 'ipynb' }

payload = urlencode(body_auth)
url = 'https://api.xbrl.us/oauth2/token'
headers = {"Content-Type": "application/x-www-form-urlencoded"}
print("payload set")
res = requests.request("POST", url, data=payload, headers=headers)
auth_json = res.json()
print('json gathtered')
if 'error' in auth_json:
    print ("\n\nThere was a problem generating an access token with these credentials. Run the first cell again to enter credentials.")
else:
    print ("\n\nYour access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. \n\nFor now, skip ahead to the section 'Make a Query'.")
access_token = auth_json['access_token']
refresh_token = auth_json['refresh_token']
newaccess = ''
newrefresh = ''
    #print('access token: ' + access_token + ' refresh token: ' + refresh_token)

payload set
json gathtered


Your access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. 

For now, skip ahead to the section 'Make a Query'.


In [80]:
#def refresh_token(): # call refresh_token() if you need to refresh
token = token if newrefresh != '' else refresh_token

refresh_auth = {'client_id': ''.join(client_id), # pulling from api connect cell
            'client_secret' : ''.join(client_secret), # pulling from api connect cell
            'grant_type' : 'refresh_token',
            'platform' : 'ipynb',
            'refresh_token' : ''.join(token) }
refreshres = requests.post(url, data=refresh_auth)
refresh_json = refreshres.json()
access_token = refresh_json['access_token']
refresh_token = refresh_json['refresh_token']
#print('access token: ' + access_token + 'refresh token: ' + refresh_token)
print('Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.')
print(access_token)

Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.
69ace430-f0cf-4148-ba6a-0f49ebc80572


***

### Defined my SP500_CIK_List here
Added to API query in the params variable

In [3]:
Wiki_SP500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') #returns multiple tables ours happens to be [0]
SP500_CIK_Array = Wiki_SP500[0]['CIK']
SP500_CIK_List = list('{:010d}'.format(n) for n in SP500_CIK_Array.values)  # Using list comprehension to return CIK codes with all leading zeroes

# I now have a pd daframe of the Wikipedia table

# SP500_CIK_List is a list datatype holding all 500 CIK Codes I need

### Defined what financial items I want returned
Added to API query in params variable

In [4]:
# Get my designated elements list                                         
GAAP_Elements = ['Revenues',                                       # Sales
                 'NetCashProvidedByUsedInOperatingActivities',     # EBITDA
                 'Assets',                                         # Assets
                 'Liabilities'                                     # Liabilties
                ]
# Note: I said I'd also pull Expenses, EBIT/DA, Accounts Receivable, D&A -- some flaws discussed in paper notes

# Data pull:
- Pull past years worth of financial info
- Write script to loop through each dict, and consolidate them into a combined dict, then combine all dicts
- Have the ability to call out whether I have missing information (achieving this by devault None values, will remove any cases where I don't have everything)

In [5]:
offset_value = 0
res_df = []

sic_code = [2080] ### v1 - OFF in API Call below

periods = ['Y']

years = [2022]  #,   # v1 - only doing 2022 and 202 while constructing... likely will variable-ize this 
         #2021,
         #2020]   # , <- put comma back
         #2019,   
         #2018,
         #2017
        # put bracket back!

string_sic = [str(int) for int in sic_code] # OFF in API URI
string_years = [str(int) for int in years]

fields = [ # this is the list of the characteristics of the data being returned by the query [COLUMNS]
         'period.fiscal-year.sort(DESC)',
         'entity.name.sort(ASC)',
         'concept.local-name.sort(ASC)', # MARK - going to try and modify this from 'local-name' TO 'is-base'
         'fact.value',
         'unit',
         'fact.decimals',
         'report.filing-date',
         'report.sic-code',             # [OFF in API URI] - 
         'entity.cik' 
        ]

params = { 
         'concept.local-name': ','.join(GAAP_Elements),                  # MARK - modified this to use my specified list of FStmt Items
         #'report.sic-code': ','.join(string_sic),                       # OFF -- spy companies
         'entity.cik': ','.join(SP500_CIK_List),                         # ON -- set to list of SP500
         'period.fiscal-year': ','.join(string_years),                   # ON -- Set to 2022 only | MARK - turn on to limit the num of yrs (currently will provide all available)
         'period.fiscal-period': ','.join(periods),
         'fact.ultimus': 'TRUE', # return only the latest occurrence of a specific fact (eg. 2018 revenues)
         'fact.has-dimensions': 'FALSE', # generally, 'FALSE' will return face financial data only
         'fields': ','.join(fields)
         }


In [6]:
# This consolidates the fields defined above, and pulls API data via a loop:

search_endpoint = 'https://api.xbrl.us/api/v1/fact/search' #looking for facts (Not reports, filers, etc.)
orig_fields = params['fields']


# Begin Loop ----------------------
count = 0
query_start = datetime.now()
printed = False
while True:
    if not printed:
        print("On", query_start.strftime("%c"), email, "(client ID:", str(client_id.split('-')[0]), "...) started the query and")  #client id defined in api connect cell
        printed = True
    res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)}) # first GET request
    res_json = res.json()
    if 'error' in res_json:
        print('There was an error: {}'.format(res_json['error_description']))
        break

    print("up to", str(offset_value + res_json['paging']['limit']), "records are found so far ...")

    res_df += res_json['data']

    if res_json['paging']['count'] < res_json['paging']['limit']: #here it's checking for whether there's more to page & printing an update
        print(" - this set contained fewer than the", res_json['paging']['limit'], "possible, only", str(res_json['paging']['count']), "records.")
        break 
    else:
        offset_value += res_json['paging']['limit'] # increments offset_value
        if 100 == res_json['paging']['limit']:
                params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)
                if offset_value == 10 * res_json['paging']['limit']:
                        break
        elif 500 == res_json['paging']['limit']:
                params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)
                if offset_value == 4 * res_json['paging']['limit']:
                        break
        params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)

if not 'error' in res_json:
    current_datetime = datetime.now().replace(microsecond=0)
    time_taken = current_datetime - query_start
    index = pd.DataFrame(res_df).index
    total_rows = len(index)
    your_limit = res_json['paging']['limit']
    limit_message = "If the results below match the limit noted above, you might not be seeing all rows, and should consider upgrading (https://xbrl.us/access-token).\n"

    if your_limit == 100:
        print("\nThis non-Member account has a limit of " , 10 * your_limit, " rows per query from our Public Filings Database. " + limit_message)
    elif your_limit == 500:
        print("\nThis Basic Individual Member account has a limit of ", 4 * your_limit, " rows per query from our Public Filings Database. " + limit_message)

    print("\nAt " + current_datetime.strftime("%c") +  ", the query finished with  ", str(total_rows), "  rows returned in " + str(time_taken) + " for \n" +  urllib.parse.unquote(res.url))


# OFF -- Dataframe creation -- (turns res_df, a list of dictionaries returned from API, and presents as a dataframe)

#    df = pd.DataFrame(res_df)
#    # the format truncates the HTML display of numerical values to two decimals; .csv data is unaffected
#    pd.options.display.float_format = '{:,.2f}'.format
#    display(HTML(df.to_html()))

On Wed Nov  8 00:52:33 2023 markstansky@gmail.com (client ID: 6a5b399f ...) started the query and
up to 500 records are found so far ...
up to 1000 records are found so far ...
up to 1500 records are found so far ...
up to 2000 records are found so far ...

This Basic Individual Member account has a limit of  2000  rows per query from our Public Filings Database. If the results below match the limit noted above, you might not be seeing all rows, and should consider upgrading (https://xbrl.us/access-token).


At Wed Nov  8 00:57:03 2023, the query finished with   2000   rows returned in 0:04:29.782239 for 
https://api.xbrl.us/api/v1/fact/search?concept.local-name=Revenues,NetCashProvidedByUsedInOperatingActivities,Assets,Liabilities&entity.cik=0000066740,0000091142,0000001800,0001551152,0001467373,0000007084,0000796343,0000008670,0000874761,0000004977,0001090872,0001559720,0000002969,0001086222,0000766421,0000915913,0001035443,0001097149,0001579241,0000352541,0000899051,0001652044,00016

# Defining my 'Factory function'
- Creating a wireframe of the dictionary I eventually want to insert into pandas

This step needs to create ~500 dictionaries to hold all SP500 company's repsective data
- Also need a unique identifier to keep track of each dictionary

In [7]:
# Initialize list to hold all my companies
all_company_financials_list_empty = []


for company in SP500_CIK_List:
    # STEP 1: create a label-list for yearxdata 
    temp_lineitem_list = []
    for year in years: #1 year
        for item in GAAP_Elements: #4 elements
            temp_lineitem_list.append(f'FY{year}_{item}') # adds year_x_data term to lineitem list
        # STEP 2: turn company yearxdata fields into a blank dictionary that holds None / NAs
        company_dictionary = {x:None for x in temp_lineitem_list}
    # STEP 3: Add Company Name and Append completed dictionary to master list
    company_dictionary['CIK_number'] = str(company) #append a unique identifier to the dictionary    
    all_company_financials_list_empty.append(company_dictionary)

## Update the Blank Factory Dict
- Lookup / Select Factory on CIK code
- Update finanicals where there is a match

In [8]:
# create 'searcher' for loop to get index of dict with matching CIK code
for dictionary in res_df: # loops same number of times as I have json dicts
    for financial_dict in all_company_financials_list_empty: # now we iterate through each dictionary we have
        if financial_dict['CIK_number'] == dictionary['entity.cik']:
            
            # update corresponding dictionary
            
            # first create temp variables
            temp_yr = dictionary['period.fiscal-year']
            temp_lclname = dictionary['concept.local-name']
            
            # then update matching line items
            if f'FY{temp_yr}_{temp_lclname}' in financial_dict: # searches current financial_dict for matching financial line item
                financial_dict[f'FY{temp_yr}_{temp_lclname}'] = dictionary['fact.value']
                
            else: 
                missing_vals.append(f'FY{temp_yr}_{temp_lclname}') # TO UPDATE: create and keep a list of missing values
            #Before exiting loop, add additional data from json dict
                ## Note: this code is inefficient -- will reassign the entity name, sic-code, filing each time a
                ## this loop is entered (when SP500's CIK list finds a match in API's entity CIK)
            # list of keys I want to append onto my custom dictionary
            extra_info = ['entity.name','report.sic-code','report.filing-date']

            for info in extra_info:
                financial_dict[info] = dictionary[info] # Adds extra api metadata that will useful for my analysis
        else: pass


# first dict will have CIK code -- use that to ID the dictionary needed

# Then go into lower loop to correclty slot datapoint into dictionary

## Throw my new-fangled data into a dataframe

In [9]:
SP500_df = pd.DataFrame(all_company_financials_list_empty) 
SP500_df

,FY2022_Revenues,FY2022_NetCashProvidedByUsedInOperatingActivities,FY2022_Assets,FY2022_Liabilities,CIK_number,entity.name,report.sic-code,report.filing-date
0,3.422900e+10,5.591000e+09,4.645500e+10,3.168500e+10,0000066740,3M COMPANY,3841.0,2023-02-08
1,NaN,3.914000e+08,3.332300e+09,1.584600e+09,0000091142,A. O. Smith Corporation,3630.0,2023-02-14
2,4.365300e+10,9.581000e+09,7.443800e+10,3.001100e+10,0000001800,ABBOTT LABORATORIES,2834.0,2023-11-01
3,5.805400e+10,2.494300e+10,1.388050e+11,1.215180e+11,0001551152,AbbVie Inc.,2834.0,2023-02-17
4,6.159430e+10,9.541129e+09,4.726339e+10,NaN,0001467373,Accenture plc,7389.0,2023-10-12
...,...,...,...,...,...,...,...,...
498,NaN,NaN,NaN,NaN,0001041061,NaN,NaN,NaN
499,NaN,NaN,NaN,NaN,0000877212,NaN,NaN,NaN
500,NaN,NaN,NaN,NaN,0001136869,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN,0000109380,NaN,NaN,NaN


In [10]:
SP500_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   FY2022_Revenues                                    259 non-null    float64
 1   FY2022_NetCashProvidedByUsedInOperatingActivities  394 non-null    float64
 2   FY2022_Assets                                      406 non-null    float64
 3   FY2022_Liabilities                                 378 non-null    float64
 4   CIK_number                                         503 non-null    object 
 5   entity.name                                        407 non-null    object 
 6   report.sic-code                                    407 non-null    float64
 7   report.filing-date                                 407 non-null    object 
dtypes: float64(5), object(3)
memory usage: 31.6+ KB


***
# Try 2: Splitting SP500 List into two
- Hopefully each time I call the API I can update the master dictionary and sequentially append each to my dictionary

In [11]:
len(SP500_CIK_List)

503

In [12]:
SPY1 = SP500_CIK_List[:250]
SPY2 = SP500_CIK_List[250:]

# API Pull 1/2 - SPY1

In [14]:
offset_value = 0
res_df = []

sic_code = [2080] ### v1 - OFF in API Call below

periods = ['Y']

years = [2022]  #,   # v1 - only doing 2022 and 202 while constructing... likely will variable-ize this 
         #2021,
         #2020]   # , <- put comma back
         #2019,   
         #2018,
         #2017
        # put bracket back!

string_sic = [str(int) for int in sic_code] # OFF in API URI
string_years = [str(int) for int in years]

fields = [ # this is the list of the characteristics of the data being returned by the query [COLUMNS]
         'period.fiscal-year.sort(DESC)',
         'entity.name.sort(ASC)',
         'concept.local-name.sort(ASC)', # MARK - going to try and modify this from 'local-name' TO 'is-base'
         'fact.value',
         'unit',
         'fact.decimals',
         'report.filing-date',
         'report.sic-code',             # [OFF in API URI] - 
         'entity.cik' 
        ]

params = { 
         'concept.local-name': ','.join(GAAP_Elements),                  # MARK - modified this to use my specified list of FStmt Items
         #'report.sic-code': ','.join(string_sic),                       # OFF -- spy companies
         'entity.cik': ','.join(SPY1),                         # ON -- set to list of SP500
         'period.fiscal-year': ','.join(string_years),                   # ON -- Set to 2022 only | MARK - turn on to limit the num of yrs (currently will provide all available)
         'period.fiscal-period': ','.join(periods),
         'fact.ultimus': 'TRUE', # return only the latest occurrence of a specific fact (eg. 2018 revenues)
         'fact.has-dimensions': 'FALSE', # generally, 'FALSE' will return face financial data only
         'fields': ','.join(fields)
         }


In [15]:
# This consolidates the fields defined above, and pulls API data via a loop:

search_endpoint = 'https://api.xbrl.us/api/v1/fact/search' #looking for facts (Not reports, filers, etc.)
orig_fields = params['fields']


# Begin Loop ----------------------
count = 0
query_start = datetime.now()
printed = False
while True:
    if not printed:
        print("On", query_start.strftime("%c"), email, "(client ID:", str(client_id.split('-')[0]), "...) started the query and")  #client id defined in api connect cell
        printed = True
    res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)}) # first GET request
    res_json = res.json()
    if 'error' in res_json:
        print('There was an error: {}'.format(res_json['error_description']))
        break

    print("up to", str(offset_value + res_json['paging']['limit']), "records are found so far ...")

    res_df += res_json['data']

    if res_json['paging']['count'] < res_json['paging']['limit']: #here it's checking for whether there's more to page & printing an update
        print(" - this set contained fewer than the", res_json['paging']['limit'], "possible, only", str(res_json['paging']['count']), "records.")
        break 
    else:
        offset_value += res_json['paging']['limit'] # increments offset_value
        if 100 == res_json['paging']['limit']:
                params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)
                if offset_value == 10 * res_json['paging']['limit']:
                        break
        elif 500 == res_json['paging']['limit']:
                params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)
                if offset_value == 4 * res_json['paging']['limit']:
                        break
        params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)

if not 'error' in res_json:
    current_datetime = datetime.now().replace(microsecond=0)
    time_taken = current_datetime - query_start
    index = pd.DataFrame(res_df).index
    total_rows = len(index)
    your_limit = res_json['paging']['limit']
    limit_message = "If the results below match the limit noted above, you might not be seeing all rows, and should consider upgrading (https://xbrl.us/access-token).\n"

    if your_limit == 100:
        print("\nThis non-Member account has a limit of " , 10 * your_limit, " rows per query from our Public Filings Database. " + limit_message)
    elif your_limit == 500:
        print("\nThis Basic Individual Member account has a limit of ", 4 * your_limit, " rows per query from our Public Filings Database. " + limit_message)

    print("\nAt " + current_datetime.strftime("%c") +  ", the query finished with  ", str(total_rows), "  rows returned in " + str(time_taken) + " for \n" +  urllib.parse.unquote(res.url))


# OFF -- Dataframe creation -- (turns res_df, a list of dictionaries returned from API, and presents as a dataframe)

#    df = pd.DataFrame(res_df)
#    # the format truncates the HTML display of numerical values to two decimals; .csv data is unaffected
#    pd.options.display.float_format = '{:,.2f}'.format
#    display(HTML(df.to_html()))

On Wed Nov  8 01:04:05 2023 markstansky@gmail.com (client ID: 6a5b399f ...) started the query and
up to 500 records are found so far ...
up to 1000 records are found so far ...
up to 1500 records are found so far ...
 - this set contained fewer than the 500 possible, only 213 records.

This Basic Individual Member account has a limit of  2000  rows per query from our Public Filings Database. If the results below match the limit noted above, you might not be seeing all rows, and should consider upgrading (https://xbrl.us/access-token).


At Wed Nov  8 01:06:45 2023, the query finished with   1213   rows returned in 0:02:39.826287 for 
https://api.xbrl.us/api/v1/fact/search?concept.local-name=Revenues,NetCashProvidedByUsedInOperatingActivities,Assets,Liabilities&entity.cik=0000066740,0000091142,0000001800,0001551152,0001467373,0000007084,0000796343,0000008670,0000874761,0000004977,0001090872,0001559720,0000002969,0001086222,0000766421,0000915913,0001035443,0001097149,0001579241,000035254

### notable that for 250 companies, it only returned 213 rows in this first batch

# Defining my 'Factory function'
- Creating a wireframe of the dictionary I eventually want to insert into pandas

## Despite running my API query in batches, this only has to be done once
- Creating frame based on all 503 SPY companies


In [16]:
# Initialize list to hold all my companies
all_company_financials_list_empty = []


for company in SP500_CIK_List:
    # STEP 1: create a label-list for yearxdata 
    temp_lineitem_list = []
    for year in years: #1 year
        for item in GAAP_Elements: #4 elements
            temp_lineitem_list.append(f'FY{year}_{item}') # adds year_x_data term to lineitem list
        # STEP 2: turn company yearxdata fields into a blank dictionary that holds None / NAs
        company_dictionary = {x:None for x in temp_lineitem_list}
    # STEP 3: Add Company Name and Append completed dictionary to master list
    company_dictionary['CIK_number'] = str(company) #append a unique identifier to the dictionary    
    all_company_financials_list_empty.append(company_dictionary)

## Update the Blank Factory Dict (for time 1/2)
- Lookup / Select Factory on CIK code
- Update finanicals where there is a match

In [17]:
# create 'searcher' for loop to get index of dict with matching CIK code
for dictionary in res_df: # loops same number of times as I have json dicts
    for financial_dict in all_company_financials_list_empty: # now we iterate through each dictionary we have
        if financial_dict['CIK_number'] == dictionary['entity.cik']:
            
            # update corresponding dictionary
            
            # first create temp variables
            temp_yr = dictionary['period.fiscal-year']
            temp_lclname = dictionary['concept.local-name']
            
            # then update matching line items
            if f'FY{temp_yr}_{temp_lclname}' in financial_dict: # searches current financial_dict for matching financial line item
                financial_dict[f'FY{temp_yr}_{temp_lclname}'] = dictionary['fact.value']
                
            else: 
                missing_vals.append(f'FY{temp_yr}_{temp_lclname}') # TO UPDATE: create and keep a list of missing values
            #Before exiting loop, add additional data from json dict
                ## Note: this code is inefficient -- will reassign the entity name, sic-code, filing each time a
                ## this loop is entered (when SP500's CIK list finds a match in API's entity CIK)
            # list of keys I want to append onto my custom dictionary
            extra_info = ['entity.name','report.sic-code','report.filing-date']

            for info in extra_info:
                financial_dict[info] = dictionary[info] # Adds extra api metadata that will useful for my analysis
        else: pass


# first dict will have CIK code -- use that to ID the dictionary needed

# Then go into lower loop to correclty slot datapoint into dictionary

***


# API Pull 2/2 - SPY2
Updating line 34 of the below code

In [19]:
offset_value = 0
res_df = []

sic_code = [2080] ### v1 - OFF in API Call below

periods = ['Y']

years = [2022]  #,   # v1 - only doing 2022 and 202 while constructing... likely will variable-ize this 
         #2021,
         #2020]   # , <- put comma back
         #2019,   
         #2018,
         #2017
        # put bracket back!

string_sic = [str(int) for int in sic_code] # OFF in API URI
string_years = [str(int) for int in years]

fields = [ # this is the list of the characteristics of the data being returned by the query [COLUMNS]
         'period.fiscal-year.sort(DESC)',
         'entity.name.sort(ASC)',
         'concept.local-name.sort(ASC)', # MARK - going to try and modify this from 'local-name' TO 'is-base'
         'fact.value',
         'unit',
         'fact.decimals',
         'report.filing-date',
         'report.sic-code',             # [OFF in API URI] - 
         'entity.cik' 
        ]

params = { 
         'concept.local-name': ','.join(GAAP_Elements),                  # MARK - modified this to use my specified list of FStmt Items
         #'report.sic-code': ','.join(string_sic),                       # OFF -- spy companies
         'entity.cik': ','.join(SPY2),                         # ON -- set to list of SP500
         'period.fiscal-year': ','.join(string_years),                   # ON -- Set to 2022 only | MARK - turn on to limit the num of yrs (currently will provide all available)
         'period.fiscal-period': ','.join(periods),
         'fact.ultimus': 'TRUE', # return only the latest occurrence of a specific fact (eg. 2018 revenues)
         'fact.has-dimensions': 'FALSE', # generally, 'FALSE' will return face financial data only
         'fields': ','.join(fields)
         }


In [20]:
# This consolidates the fields defined above, and pulls API data via a loop:

search_endpoint = 'https://api.xbrl.us/api/v1/fact/search' #looking for facts (Not reports, filers, etc.)
orig_fields = params['fields']


# Begin Loop ----------------------
count = 0
query_start = datetime.now()
printed = False
while True:
    if not printed:
        print("On", query_start.strftime("%c"), email, "(client ID:", str(client_id.split('-')[0]), "...) started the query and")  #client id defined in api connect cell
        printed = True
    res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)}) # first GET request
    res_json = res.json()
    if 'error' in res_json:
        print('There was an error: {}'.format(res_json['error_description']))
        break

    print("up to", str(offset_value + res_json['paging']['limit']), "records are found so far ...")

    res_df += res_json['data']

    if res_json['paging']['count'] < res_json['paging']['limit']: #here it's checking for whether there's more to page & printing an update
        print(" - this set contained fewer than the", res_json['paging']['limit'], "possible, only", str(res_json['paging']['count']), "records.")
        break 
    else:
        offset_value += res_json['paging']['limit'] # increments offset_value
        if 100 == res_json['paging']['limit']:
                params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)
                if offset_value == 10 * res_json['paging']['limit']:
                        break
        elif 500 == res_json['paging']['limit']:
                params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)
                if offset_value == 4 * res_json['paging']['limit']:
                        break
        params['fields'] = orig_fields + ',fact.offset({})'.format(offset_value)

if not 'error' in res_json:
    current_datetime = datetime.now().replace(microsecond=0)
    time_taken = current_datetime - query_start
    index = pd.DataFrame(res_df).index
    total_rows = len(index)
    your_limit = res_json['paging']['limit']
    limit_message = "If the results below match the limit noted above, you might not be seeing all rows, and should consider upgrading (https://xbrl.us/access-token).\n"

    if your_limit == 100:
        print("\nThis non-Member account has a limit of " , 10 * your_limit, " rows per query from our Public Filings Database. " + limit_message)
    elif your_limit == 500:
        print("\nThis Basic Individual Member account has a limit of ", 4 * your_limit, " rows per query from our Public Filings Database. " + limit_message)

    print("\nAt " + current_datetime.strftime("%c") +  ", the query finished with  ", str(total_rows), "  rows returned in " + str(time_taken) + " for \n" +  urllib.parse.unquote(res.url))


# OFF -- Dataframe creation -- (turns res_df, a list of dictionaries returned from API, and presents as a dataframe)

#    df = pd.DataFrame(res_df)
#    # the format truncates the HTML display of numerical values to two decimals; .csv data is unaffected
#    pd.options.display.float_format = '{:,.2f}'.format
#    display(HTML(df.to_html()))

On Wed Nov  8 01:11:50 2023 markstansky@gmail.com (client ID: 6a5b399f ...) started the query and
up to 500 records are found so far ...
up to 1000 records are found so far ...
up to 1500 records are found so far ...
 - this set contained fewer than the 500 possible, only 279 records.

This Basic Individual Member account has a limit of  2000  rows per query from our Public Filings Database. If the results below match the limit noted above, you might not be seeing all rows, and should consider upgrading (https://xbrl.us/access-token).


At Wed Nov  8 01:12:23 2023, the query finished with   1279   rows returned in 0:00:32.132243 for 
https://api.xbrl.us/api/v1/fact/search?concept.local-name=Revenues,NetCashProvidedByUsedInOperatingActivities,Assets,Liabilities&entity.cik=0001110803,0000879169,0001699150,0001145197,0000050863,0001571949,0000051253,0000051434,0000051644,0000896878,0001035267,0000914208,0001687229,0001478242,0001020569,0000728535,0000779152,0000052988,0000200406,000083344

## Update the 50% filled Factory Dict (for time 2/2)
- Just need to run this code block a second time to find and place the remaining values

In [21]:
# create 'searcher' for loop to get index of dict with matching CIK code
for dictionary in res_df: # loops same number of times as I have json dicts
    for financial_dict in all_company_financials_list_empty: # now we iterate through each dictionary we have
        if financial_dict['CIK_number'] == dictionary['entity.cik']:
            
            # update corresponding dictionary
            
            # first create temp variables
            temp_yr = dictionary['period.fiscal-year']
            temp_lclname = dictionary['concept.local-name']
            
            # then update matching line items
            if f'FY{temp_yr}_{temp_lclname}' in financial_dict: # searches current financial_dict for matching financial line item
                financial_dict[f'FY{temp_yr}_{temp_lclname}'] = dictionary['fact.value']
                
            else: 
                missing_vals.append(f'FY{temp_yr}_{temp_lclname}') # TO UPDATE: create and keep a list of missing values
            #Before exiting loop, add additional data from json dict
                ## Note: this code is inefficient -- will reassign the entity name, sic-code, filing each time a
                ## this loop is entered (when SP500's CIK list finds a match in API's entity CIK)
            # list of keys I want to append onto my custom dictionary
            extra_info = ['entity.name','report.sic-code','report.filing-date']

            for info in extra_info:
                financial_dict[info] = dictionary[info] # Adds extra api metadata that will useful for my analysis
        else: pass


# first dict will have CIK code -- use that to ID the dictionary needed

# Then go into lower loop to correclty slot datapoint into dictionary

In [22]:
SP500_df = pd.DataFrame(all_company_financials_list_empty) 
SP500_df

,FY2022_Revenues,FY2022_NetCashProvidedByUsedInOperatingActivities,FY2022_Assets,FY2022_Liabilities,CIK_number,entity.name,report.sic-code,report.filing-date
0,3.422900e+10,5.591000e+09,46455000000,3.168500e+10,0000066740,3M COMPANY,3841,2023-02-08
1,NaN,3.914000e+08,3332300000,1.584600e+09,0000091142,A. O. Smith Corporation,3630,2023-02-14
2,4.365300e+10,9.581000e+09,74438000000,3.001100e+10,0000001800,ABBOTT LABORATORIES,2834,2023-11-01
3,5.805400e+10,2.494300e+10,138805000000,1.215180e+11,0001551152,AbbVie Inc.,2834,2023-02-17
4,6.159430e+10,9.541129e+09,47263390000,NaN,0001467373,Accenture plc,7389,2023-10-12
...,...,...,...,...,...,...,...,...
498,6.842000e+09,1.427000e+09,5846000000,1.472200e+10,0001041061,"YUM! BRANDS, INC.",5812,2023-02-27
499,5.781000e+09,4.880000e+08,7529000000,4.796000e+09,0000877212,Zebra Technologies Corporation,3560,2023-02-16
500,NaN,NaN,21066000000,9.039000e+09,0001136869,"ZIMMER BIOMET HOLDINGS, INC.",3842,2023-05-02
501,6.140000e+08,1.470000e+09,89545000000,8.465200e+10,0000109380,"ZIONS BANCORPORATION, NATIONAL ASSOCIATION",6021,2023-02-23


# LETS GOOOOOOOOOO

- Going to explore it quickly then will export to csv

In [23]:
SP500_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   FY2022_Revenues                                    324 non-null    float64
 1   FY2022_NetCashProvidedByUsedInOperatingActivities  487 non-null    float64
 2   FY2022_Assets                                      503 non-null    int64  
 3   FY2022_Liabilities                                 472 non-null    float64
 4   CIK_number                                         503 non-null    object 
 5   entity.name                                        503 non-null    object 
 6   report.sic-code                                    503 non-null    int64  
 7   report.filing-date                                 503 non-null    object 
dtypes: float64(3), int64(2), object(3)
memory usage: 31.6+ KB


I find it interesting that I have full data in 5/8 columns.  The columns where I don't include Revenue, Cash From Ops, and Liabilities.


I had trouble with several Revenue cases as I was builing the API, it had to do with the type of company and how they labeled revenue.  I will need to address this in Sprint 3.

***
***
# Cleaning so only the complete rows come to the next step in analysis

In [ ]:
['Chapter', 'SICDivision', 'SICIndustryGroup','SICMajGroup','SICPrimary','Voluntary','YearFiled','Date10k1Before',
 'Sales1Before','Sales1Before','EbitBefore','Assets1Before','Liab1Before']

In [26]:
spy_ordered = ['CIK_number','entity.name','report.sic-code','report.filing-date','FY2022_Revenues',
               'FY2022_NetCashProvidedByUsedInOperatingActivities','FY2022_Assets','FY2022_Liabilities']
               
SP500_df_ordered = SP500_df[spy_ordered]

In [27]:
SP500_df_ordered.head()

,CIK_number,entity.name,report.sic-code,report.filing-date,FY2022_Revenues,FY2022_NetCashProvidedByUsedInOperatingActivities,FY2022_Assets,FY2022_Liabilities
0,0000066740,3M COMPANY,3841,2023-02-08,3.422900e+10,5.591000e+09,46455000000,3.168500e+10
1,0000091142,A. O. Smith Corporation,3630,2023-02-14,NaN,3.914000e+08,3332300000,1.584600e+09
2,0000001800,ABBOTT LABORATORIES,2834,2023-11-01,4.365300e+10,9.581000e+09,74438000000,3.001100e+10
3,0001551152,AbbVie Inc.,2834,2023-02-17,5.805400e+10,2.494300e+10,138805000000,1.215180e+11
4,0001467373,Accenture plc,7389,2023-10-12,6.159430e+10,9.541129e+09,47263390000,NaN


In [31]:
# filtering out nulls in the financials
fin_cols=['FY2022_Revenues','FY2022_NetCashProvidedByUsedInOperatingActivities','FY2022_Assets','FY2022_Liabilities']
SP500_df_export = SP500_df_ordered[SP500_df_ordered[fin_cols].notnull().all(1)]

### 303 columns are less than what I wanted but I will address those non-revenue cases in Sprint 3

In [32]:
# To CSV
SP500_df_export.to_csv('/Users/markstansky/Desktop/Brainstation/Capstone/Sprint_2/Submission/sp500_api_financials_v1.csv')